<a href="https://colab.research.google.com/github/OmerRosen/Kaggle/blob/main/IMDB_2022_Hit_or_Flop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB 2022 - Hit or Flop

## Mission Statement

### Data To Collect

#### Search box page:
* Movie Id
* Movie Title
* Movie year
* Rating
* MetaScore - Outcome
* Description
* Poster
* Directore name + Link
* Stars
* Votes - Output
* Genre

#### Main Movie Page data
* Movie Length
* Rank
* Writer
* Star #1
* Star #2
* Star #3
* User eviews
* Critic reviews
* Number of photos posted
* Storyline - Text
* Tag line
* Release Date - Month
* Country of origin
* Additional Lanaguages
* Country of filming
* Production companies
* Budget
* Opening Weekend Date
* Gross US & Canada - Output
* Opening weekend US & Canada - Output
* Gross worldwide - Output
* Color - Color
* Color - Black&White
* Sound mix - Dolby Digital
* Sound mix - Dolby Atmos
* Aspect ratio

#### Director Page:
* Director Age
* Is Top 500?
* Director Gender (Based on bio)
* Previous film count - As Director
* Previous film count - As Writer
* Previous film count - As As Producer
* Director Publicity listing count

#### Information Abount Cast:
* Full list of cast and their profile links
* Number of cast members
* Produced by - Is top 500?
* Music by - Is top 500?
* Cinematography by  - Is top 500
* Film Editing by   - Is top 500
* Art Direction by  - Is top 500
* Number of Production Management
* Number of Art Department
* Number of Sound Department
* Number of Camera and Electrical Department
* Number of Editorial Department
* Number of Music Department
* Number of Additional Crew

#### Actor Page
* Is Top 5000?
* Is Top 500?
* Is Top 100 (aka - Has numberical rank)
* Is Top 10 (aka - Has numberical rank)
* Numerical Rank (Could be none)
* Gender
* Age
* Oscar nominations 
* Birth country

#### Aggragated data:
* How many male stars
* How many female stars
* Avrage cast memeber age
* Max/Min age of cast member
* Number of unique birth countries of actors
* Total number of Oscar numinatior for cast
* Num cast members in top 5000
* Num cast memebers in top 500
* Num cast members in top 100
* Num cast members in top 10

# IMDB Scraper

## Import Libraries

In [ ]:
import requests
from IPython.core.display import HTML
import json
from time import sleep
import re
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import traceback


## Set main variables

In [ ]:
base_url = "https://www.imdb.com"
base_folder_path = "/content/drive/My Drive/Harvard HW/Course 4 - Final Project"

start_date = "2022-01-01"
end_date = "2022-12-31"
minimum_votes = 10 # Minimum vote amount to coolect movie
start_point = 1 # Start from movie #1-50

movie_search_url = f"{base_url}/search/title/?title_type=feature&release_date={start_date},{end_date}&num_votes={minimum_votes},&{start_point}anguages=en&start=1&ref_=adv_nxt"
movie_search_url

'https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=10,&1anguages=en&start=1&ref_=adv_nxt'

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

list_of_50_movies = requests.get(f'{movie_search_url}', headers=headers, timeout=10)
list_of_50_movies

<Response [200]>

In [ ]:
list_of_50_movies_soup = BeautifulSoup(list_of_50_movies.text, 'html.parser').find_all('div',{'class':'lister-item mode-advanced'})


In [ ]:
movie_search_page = list_of_50_movies_soup[0]

## Main Functions:

### Get Movie Box details

In [ ]:
def get_basic_details(movie_search_page):
  search_box_info = {}
  search_box_info['movie_name'] = movie_search_page.find('a')
  search_box_info['movie_page_url'] = f"{base_url}{movie_search_page.find('a')['href']}"

  try:
    search_box_info['movie_place'] = None if not movie_search_page.find('span',{'class':'lister-item-index unbold text-primary'}) else movie_search_page.find('span',{'class':'lister-item-index unbold text-primary'}).text.replace('.','')
    search_box_info['movie_id'] = None if not search_box_info['movie_page_url'] else search_box_info['movie_page_url'].split('/')[-2]
    search_box_info['movie_name'] = None if not movie_search_page.find('a').find('img') else movie_search_page.find('a').find('img')['alt'].strip()
    search_box_info['movie_thubmnail'] = None if not movie_search_page.find('a').find('img') else movie_search_page.find('a').find('img')['src']
    #search_box_info['movie_year'] = None if not movie_search_page.find('span',{'class':'lister-item-year text-muted unbold'}) else movie_search_page.find('span',{'class':'lister-item-year text-muted unbold'}).text.replace('(','').replace(')','')
    #search_box_info['movie_rating'] = None if not movie_search_page.find('strong') else movie_search_page.find('strong').text
    search_box_info['movie_metascore'] = None if not movie_search_page.find('span', {'class': 'metascore'}) else movie_search_page.find('span', {'class': 'metascore'}).text.strip()
    search_box_info['movie_description'] = None if not movie_search_page.find_all('p',{'class':'text-muted'}) else movie_search_page.find_all('p',{'class':'text-muted'})[1].text.strip()
    search_box_info['runtime_min'] = None if not movie_search_page.find('span',{'class':'runtime'}) else movie_search_page.find('span',{'class':'runtime'}).text.split(' ')[0]

    bottom_box_info = movie_search_page.find('p',{'class':'sort-num_votes-visible'}).find_all('span')
    search_box_info['movie_vote_num'] = bottom_box_info[1]['data-value']
    search_box_info['movie_gross'] = bottom_box_info[4]['data-value'] if len(bottom_box_info)>2 else None;

    search_box_info['movie_rating'] = None if not movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'certificate'}) else movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'certificate'}).text.strip()
    search_box_info['movie_genere'] = None if not movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'genre'}) else movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'genre'}).text.strip()

    search_box_info['__SuccsefullyCollectBasicDetails'] = True;

  except Exception as e:
    print(f"Failed extracting data for movie: {search_box_info['movie_name']}. \nUrl: {search_box_info['movie_page_url']}.\n Error:\n{e}")
    traceback.print_exc()
    search_box_info['__SuccsefullyCollectBasicDetails'] = False;


  return search_box_info

In [ ]:
movie_search_page = list_of_50_movies_soup[1]
search_box_info = get_basic_details(movie_search_page)
search_box_info



{'movie_name': 'Everything Everywhere All at Once',
 'movie_page_url': 'https://www.imdb.com/title/tt6710474/?ref_=adv_li_i',
 'movie_place': '2',
 'movie_id': 'tt6710474',
 'movie_thubmnail': 'https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png',
 'movie_metascore': '81',
 'movie_description': 'A middle-aged Chinese immigrant is swept up into an insane adventure in which she alone can save existence by exploring other universes and connecting with the lives she could have led.',
 'runtime_min': '139',
 'movie_vote_num': '415086',
 'movie_gross': '72,861,480',
 'movie_rating': 'R',
 'movie_genere': 'Action, Adventure, Comedy',
 '__SuccsefullyCollectBasicDetails': True}

In [ ]:
movie_main_page = requests.get(search_box_info['movie_page_url'], headers=headers)
movie_main_page_soup = BeautifulSoup(movie_main_page.text, 'html.parser')

### Extract Movie Artists

In [ ]:
def extract_artist(artist_type, movie_main_page_soup):

  item_dict = {}

  search_result = movie_main_page_soup.find(string=f"{artist_type}s") if movie_main_page_soup.find(string=artist_type) is None else movie_main_page_soup.find(string=artist_type)
  search_item_list = []
  if search_result!=None:
    search_item_list = search_result.find_parent().find_next_sibling().find_all('a')
  
  item_dict[f"{artist_type}_count"] = len(search_item_list)

  for i,item in enumerate(search_item_list):
    artisc_name = item.text.strip()
    artisc_url = item['href']
    artist_id = artisc_url.split('/')[2]
    item_dict[f"{artist_type}_{i+1}_name"] = artisc_name
    #item_dict[f"{artist_type}_{i+1}_url"] = artisc_url
    item_dict[f"{artist_type}_{i+1}_imdb_id"] = artist_id

  return item_dict

print(extract_artist("Director", movie_main_page_soup))
print(extract_artist("Writer", movie_main_page_soup))
print(extract_artist("Star", movie_main_page_soup))

{'Director_count': 2, 'Director_1_name': 'Daniel Kwan', 'Director_1_imdb_id': 'nm3453283', 'Director_2_name': 'Daniel Scheinert', 'Director_2_imdb_id': 'nm3215397'}
{'Writer_count': 2, 'Writer_1_name': 'Daniel Kwan', 'Writer_1_imdb_id': 'nm3453283', 'Writer_2_name': 'Daniel Scheinert', 'Writer_2_imdb_id': 'nm3215397'}
{'Star_count': 3, 'Star_1_name': 'Michelle Yeoh', 'Star_1_imdb_id': 'nm0000706', 'Star_2_name': 'Stephanie Hsu', 'Star_2_imdb_id': 'nm3513533', 'Star_3_name': 'Jamie Lee Curtis', 'Star_3_imdb_id': 'nm0000130'}


### Perform Currency Conversion

In [ ]:
currency_codes = {
    "$":"USD",
    "€":"EUR",
    "£":"GBP",
    "₹":"INR",
    "â‚¬": "EUR",
    "â‚¹": "INR",
    "CA$": "CAD",
    "NOKÂ": "NOK",
    "Â£": "GBP",
    "CHF": "CHF",
    "Â¥": "JPY",
    "PKR": "PKR",
    "A$": "AUD",
    "CZK": "CZK",
    "RUR": "RUB",
    "NZ$": "NZD",
    "MYR": "MYR",
    "NGN": "NGN",
    "NOK":"NOK"
}


currency_conversion_values = {}


In [ ]:
def extract_currency_symbol(s):
    match = re.search(r"[^\d]+", s)
    if match:
        return match.group(0).strip()
    else:
        return ""

In [ ]:
def get_currency_code(amount_string):
  currency_symbol = extract_currency_symbol(amount_string)
  if currency_symbol in currency_codes:
      return currency_codes[currency_symbol]
  else:
    print(f"Could not find a value for: {currency_symbol} in {amount_string}")
    return None

In [ ]:
def extract_numerical_value(string):
    # Remove all non-numeric characters from the string
    numerical_string = re.sub(r"[^\d.]+", "", string)
    # Convert the string to a float and return it
    return float(numerical_string)

In [ ]:

with open(f'{base_folder_path}/openexchangerates.txt', 'r') as f:
    api_key = f.read().strip()

api_key

def convert_string_amount_to_usd(amount):

  # get the currency code and amount value
  currency_code = get_currency_code(amount)
  original_amount = extract_numerical_value(amount)
  usd_amount = None

  if currency_code==None:
    usd_amount=None
  elif currency_code!="USD":
  
    if currency_code in currency_conversion_values:
      exchange_rate = currency_conversion_values[currency_code]
    else:
      # make API request to get exchange rate for the currency code
      url = f"https://openexchangerates.org/api/latest.json?app_id={api_key}&symbols={currency_code}"
      print(amount,currency_code,original_amount,url)
      response = requests.get(url)
      print(response)
      # parse the exchange rate from the API response
      exchange_rate = response.json()["rates"][currency_code]
      
      currency_conversion_values[currency_code] = exchange_rate
      
      # calculate the USD equivalent amount
      usd_amount = round(original_amount / exchange_rate)
      
      print(f"{amount} {currency_code} = {usd_amount} USD")
  else:
    usd_amount = original_amount

  return usd_amount,original_amount,currency_code



convert_string_amount_to_usd("CA$15,000")
convert_string_amount_to_usd("NOKÂ 80,200,000")
convert_string_amount_to_usd("NOKÂ 20,000,000")
convert_string_amount_to_usd("â‚¹3,500,000,000")
currency_conversion_values

CA$15,000 CAD 15000.0 https://openexchangerates.org/api/latest.json?app_id=ac63294868f6455a91a128e22f4d9f35&symbols=CAD
<Response [200]>
CA$15,000 CAD = 11155 USD
NOKÂ 80,200,000 NOK 80200000.0 https://openexchangerates.org/api/latest.json?app_id=ac63294868f6455a91a128e22f4d9f35&symbols=NOK
<Response [200]>
NOKÂ 80,200,000 NOK = 7780520 USD
â‚¹3,500,000,000 INR 3500000000.0 https://openexchangerates.org/api/latest.json?app_id=ac63294868f6455a91a128e22f4d9f35&symbols=INR
<Response [200]>
â‚¹3,500,000,000 INR = 42584003 USD


{'CAD': 1.34474, 'NOK': 10.307795, 'INR': 82.190488}

### Get Movie Page Details

In [ ]:
def get_extended_details(movie_page_url, movie_name):
  movie_page_dict = {}

  try:

    movie_main_page = requests.get(movie_page_url, headers=headers)
    movie_main_page_soup = BeautifulSoup(movie_main_page.text, 'html.parser')   

    year_rating = movie_main_page_soup.find('ul',{"class":"ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt"}).find_all('a')

    movie_page_dict['movie_year'] = year_rating[0].text if len(year_rating)>0 else None
    movie_page_dict['movie_rating'] = year_rating[1].text if len(year_rating)>1 else None

    director_dict = extract_artist("Director", movie_main_page_soup)
    if director_dict[f"Director_count"] == 0:
      print(f"No Director found for movie {movie_name}. Url: {movie_page_url}")
    movie_page_dict.update(director_dict)

    writer_dict = extract_artist("Writer", movie_main_page_soup)
    if writer_dict[f"Writer_count"] == 0:
      print(f"No Writer found for movie {movie_name}. Url: {movie_page_url}")
    movie_page_dict.update(writer_dict)

    star_dict = extract_artist("Star", movie_main_page_soup)
    if star_dict[f"Star_count"] == 0:
      print(f"No Star found for movie {movie_name}. Url: {movie_page_url}")
    movie_page_dict.update(star_dict)

    review_scores = movie_main_page_soup.find_all('span',{'class':'score'})
    movie_page_dict['user_reviews_count'] = review_scores[0].text if len(review_scores)>0 else None
    movie_page_dict['critic_reviews_count'] = review_scores[1].text if len(review_scores)>1 else None

    movie_page_dict['release_date'] = movie_main_page_soup.find('a',{'class':'ipc-metadata-list-item__label ipc-metadata-list-item__label--link'}, string="Release date").find_parent().find('a',{'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'}).text.split('(')[0].strip()
    movie_page_dict['release_date']

    movie_page_dict['budget'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Budget")
    if (movie_page_dict['budget']!=None):
      movie_page_dict['budget'] = movie_page_dict['budget'].find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

      usd_amount,original_amount,currency_code = convert_string_amount_to_usd(movie_page_dict['budget'])
      movie_page_dict['budget_usd'] = usd_amount
      movie_page_dict['budget_currency'] = currency_code

    movie_page_dict['gross_worldwide'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Gross worldwide")
    if (movie_page_dict['gross_worldwide']!=None):
      movie_page_dict['gross_worldwide'] = movie_page_dict['gross_worldwide'].find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

    movie_page_dict['gross_us_canada'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Gross US & Canada")
    if (movie_page_dict['gross_us_canada']!=None):
      movie_page_dict['gross_us_canada'] = movie_page_dict['gross_us_canada'].find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

    movie_page_dict['opening_weekend_us_canada'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Opening weekend US & Canada")
    if (movie_page_dict['opening_weekend_us_canada']!=None):
      movie_page_dict['opening_weekend_us_canada'] = movie_page_dict['opening_weekend_us_canada'].find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

    movie_page_dict['origin_country'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Country of origin")
    if (movie_page_dict['origin_country']!=None):
      movie_page_dict['origin_country'] = movie_page_dict['origin_country'].find_next_sibling().find('a').text.split('(')[0].strip()

    languages = movie_main_page_soup.find('span',string="Language") if movie_main_page_soup.find('span',string="Languages") is None else movie_main_page_soup.find('span',string="Languages")
    languages = [] if not languages else languages.find_next_sibling().find_all('a')
    movie_page_dict['languages'] = ",".join([language.text for language in languages])

    movie_page_dict['__SuccsefullyCollectExtandedDetails'] = True;

    

  except Exception as e:
    print(f"Failed extracting data for movie: {movie_name}. \nUrl: {movie_page_url}.\n Error:\n{e}")
    traceback.print_exc()
    movie_page_dict['__SuccsefullyCollectExtandedDetails'] = False;

  return movie_page_dict


get_extended_details(search_box_info['movie_page_url'],search_box_info['movie_name'])

{'movie_year': '2022',
 'movie_rating': 'R',
 'Director_count': 2,
 'Director_1_name': 'Daniel Kwan',
 'Director_1_imdb_id': 'nm3453283',
 'Director_2_name': 'Daniel Scheinert',
 'Director_2_imdb_id': 'nm3215397',
 'Writer_count': 2,
 'Writer_1_name': 'Daniel Kwan',
 'Writer_1_imdb_id': 'nm3453283',
 'Writer_2_name': 'Daniel Scheinert',
 'Writer_2_imdb_id': 'nm3215397',
 'Star_count': 3,
 'Star_1_name': 'Michelle Yeoh',
 'Star_1_imdb_id': 'nm0000706',
 'Star_2_name': 'Stephanie Hsu',
 'Star_2_imdb_id': 'nm3513533',
 'Star_3_name': 'Jamie Lee Curtis',
 'Star_3_imdb_id': 'nm0000130',
 'user_reviews_count': '3.3K',
 'critic_reviews_count': '395',
 'release_date': 'April 8, 2022',
 'budget': '$14,300,000',
 'budget_usd': 14300000.0,
 'budget_currency': 'USD',
 'gross_worldwide': '$136,137,373',
 'gross_us_canada': '$77,020,896',
 'opening_weekend_us_canada': '$501,305',
 'origin_country': 'United States',
 'languages': 'English,Mandarin,Cantonese',
 '__SuccsefullyCollectExtandedDetails': T

In [ ]:
# print(search_box_info['movie_page_url'],search_box_info['movie_name'])
# cast_url = movie_main_page_soup.find('a',{'class':'ipc-metadata-list-item__label ipc-metadata-list-item__label--link'})['href']
# cast_url = f"{base_url}{cast_url}"
# cast_url

## Cursor for collecting movies in batches

In [ ]:
# Fixed parameters:
base_url = "https://www.imdb.com"

start_date = "2022-01-01"
end_date = "2022-12-31"
minimum_votes = 20 # Minimum vote amount to coolect movie

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}

In [ ]:
# Dynamic parameters:

start_point = 1 # Start from movie #1-50

movie_search_url_50_batch = f"{base_url}/search/title/?title_type=feature&release_date={start_date},{end_date}&num_votes={minimum_votes},&user_rating=1.0,10.0&countries=us&languages=en&start={start_point}&ref_=adv_nxt"
print(f"movie_search_url_50_batch: {movie_search_url_50_batch}")
search_page = requests.get(f'{movie_search_url_50_batch}', headers=headers, timeout=10)
search_page_soup = BeautifulSoup(list_of_50_movies.text, 'html.parser')
list_of_50_movies_soup = search_page_soup.find_all('div',{'class':'lister-item mode-advanced'})

max_num_of_results = search_page_soup.find('div',{'class':'desc'}).find('span').text.split(' ')[2].replace(",", "")
max_num_of_results = int(max_num_of_results)

print(f"max_num_of_results: {max_num_of_results}")

movie_search_url_50_batch: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&user_rating=1.0,10.0&countries=us&languages=en&start=1&ref_=adv_nxt
max_num_of_results: 6306


In [ ]:
search_page = requests.get(f'{movie_search_url_50_batch}', headers=headers, timeout=10)
search_page_soup = BeautifulSoup(search_page.text, 'html.parser')
list_of_50_movies_soup = search_page_soup.find_all('div',{'class':'lister-item mode-advanced'})

max_num_of_results = search_page_soup.find('div',{'class':'desc'}).find('span').text.split(' ')[2].replace(",", "")
max_num_of_results = int(max_num_of_results)
max_num_of_results

1384

In [ ]:
total_movie_dataset = []
total_actor_dataset = []
total_director_dataset = []

start_point = 1 # Start from movie #1-50
while start_point < max_num_of_results-50:

  movie_search_url_50_batch = f"{base_url}/search/title/?title_type=feature&release_date={start_date},{end_date}&num_votes={minimum_votes},&languages=en&start={start_point}&ref_=adv_nxt"
  print(f"movie_search_url_50_batch start point {start_point}: {movie_search_url_50_batch}")
  search_page = requests.get(f'{movie_search_url_50_batch}', headers=headers, timeout=10)
  search_page_soup = BeautifulSoup(search_page.text, 'html.parser')
  list_of_50_movies_soup = search_page_soup.find_all('div',{'class':'lister-item mode-advanced'})

  for i,movie in enumerate(list_of_50_movies_soup):
    movie_data = {}

    basic_data = get_basic_details(movie)
    movie_data.update(basic_data)

    if basic_data['__SuccsefullyCollectBasicDetails']==True:
      movie_main_page = requests.get(basic_data['movie_page_url'], headers=headers)
      movie_main_page_soup = BeautifulSoup(movie_main_page.text, 'html.parser')   

      if i%50==0:
        print(basic_data['movie_name'],basic_data['movie_page_url'])

      extended_deta = get_extended_details(basic_data['movie_page_url'],basic_data['movie_name'])
      movie_data.update(extended_deta)

      sleep(0.1)

      total_movie_dataset.append(movie_data)
    else:
      movie_data['__SuccsefullyCollectExtandedDetails'] = False;
      print(f"Skipping extanded data for: {basic_data['movie_name']} - {basic_data['movie_page_url']}")

  movie_dataset = pd.DataFrame(total_movie_dataset)
  movie_dataset.to_csv("/content/drive/My Drive/Harvard HW/Course 4 - Final Project/total_movie_dataset.csv", index=False)

  start_point += 50

movie_search_url_50_batch start point 1: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=1&ref_=adv_nxt
Avatar: The Way of Water https://www.imdb.com/title/tt1630029/?ref_=adv_li_i
€10,000,000 EUR 10000000.0 https://openexchangerates.org/api/latest.json?app_id=ac63294868f6455a91a128e22f4d9f35&symbols=EUR
<Response [200]>
€10,000,000 EUR = 10954702 USD
movie_search_url_50_batch start point 51: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=51&ref_=adv_nxt
Terrifier 2 https://www.imdb.com/title/tt10403420/?ref_=adv_li_i
movie_search_url_50_batch start point 101: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=101&ref_=adv_nxt
The Devil Conspiracy https://www.imdb.com/title/tt9663168/?ref_=adv_li_i
movie_search_url_50_batch start point 151: https://www.imdb.com/search/t

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/urllib3/response.py", line 761, in _update_chunk_length
    self.chunk_left = int(line, 16)
ValueError: invalid literal for int() with base 16: b''

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/urllib3/response.py", line 444, in _error_catcher
    yield
  File "/usr/local/lib/python3.9/dist-packages/urllib3/response.py", line 828, in read_chunked
    self._update_chunk_length()
  File "/usr/local/lib/python3.9/dist-packages/urllib3/response.py", line 765, in _update_chunk_length
    raise InvalidChunkLength(self, line)
urllib3.exceptions.InvalidChunkLength: InvalidChunkLength(got length b'', 0 bytes read)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/requests/models.py", line 760, in generate
    

movie_search_url_50_batch start point 301: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=301&ref_=adv_nxt
Play Dead https://www.imdb.com/title/tt20198774/?ref_=adv_li_i
CHF 2,600,000 CHF 2600000.0 https://openexchangerates.org/api/latest.json?app_id=ac63294868f6455a91a128e22f4d9f35&symbols=CHF
<Response [200]>
CHF 2,600,000 CHF = 2870102 USD
movie_search_url_50_batch start point 351: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=351&ref_=adv_nxt
Somewhere in Queens https://www.imdb.com/title/tt14097180/?ref_=adv_li_i
Could not find a value for: ¥ in ¥900,000,000
movie_search_url_50_batch start point 401: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=401&ref_=adv_nxt
Moloch https://www.imdb.com/title/tt14412446/?ref_=adv_li_i
movie_search_url_50_batch start point

Traceback (most recent call last):
  File "<ipython-input-15-b6ce4ab8c4fd>", line 24, in get_extended_details
    star_dict = extract_artist("Star", movie_main_page_soup)
  File "<ipython-input-9-5bc69bceb996>", line 8, in extract_artist
    search_item_list = search_result.find_parent().find_next_sibling().find_all('a')
AttributeError: 'NoneType' object has no attribute 'find_all'


No Writer found for movie Puppet Master: Doktor Death. Url: https://www.imdb.com/title/tt21114402/?ref_=adv_li_i
movie_search_url_50_batch start point 501: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=501&ref_=adv_nxt
Tell It Like a Woman https://www.imdb.com/title/tt5850650/?ref_=adv_li_i
movie_search_url_50_batch start point 551: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=551&ref_=adv_nxt
Mothman https://www.imdb.com/title/tt17157786/?ref_=adv_li_i
PKR 700,000,000 PKR 700000000.0 https://openexchangerates.org/api/latest.json?app_id=ac63294868f6455a91a128e22f4d9f35&symbols=PKR
<Response [200]>
PKR 700,000,000 PKR = 2435772 USD
movie_search_url_50_batch start point 601: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=601&ref_=adv_nxt
The Almond and the Seahors

Traceback (most recent call last):
  File "<ipython-input-15-b6ce4ab8c4fd>", line 9, in get_extended_details
    year_rating = movie_main_page_soup.find('ul',{"class":"ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt"}).find_all('a')
AttributeError: 'NoneType' object has no attribute 'find_all'


movie_search_url_50_batch start point 701: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=701&ref_=adv_nxt
Don't Look at the Demon https://www.imdb.com/title/tt5249706/?ref_=adv_li_i
A$640,000 AUD 640000.0 https://openexchangerates.org/api/latest.json?app_id=ac63294868f6455a91a128e22f4d9f35&symbols=AUD
<Response [200]>
A$640,000 AUD = 431672 USD
movie_search_url_50_batch start point 751: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=751&ref_=adv_nxt
The Hunted https://www.imdb.com/title/tt16102916/?ref_=adv_li_i
movie_search_url_50_batch start point 801: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=801&ref_=adv_nxt
Manifest West https://www.imdb.com/title/tt12164416/?ref_=adv_li_i
No Writer found for movie Strange Love. Url: https://www.imdb.com/title/tt16755700

Traceback (most recent call last):
  File "<ipython-input-15-b6ce4ab8c4fd>", line 24, in get_extended_details
    star_dict = extract_artist("Star", movie_main_page_soup)
  File "<ipython-input-9-5bc69bceb996>", line 8, in extract_artist
    search_item_list = search_result.find_parent().find_next_sibling().find_all('a')
AttributeError: 'NoneType' object has no attribute 'find_all'


movie_search_url_50_batch start point 1301: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=1301&ref_=adv_nxt
Albatross https://www.imdb.com/title/tt12992082/?ref_=adv_li_i
No Star found for movie Home2Home. Url: https://www.imdb.com/title/tt18304232/?ref_=adv_li_i
No Writer found for movie Day of Disappearance. Url: https://www.imdb.com/title/tt16341768/?ref_=adv_li_i


In [ ]:
movie_dataset = pd.DataFrame(total_movie_dataset)
movie_dataset.to_csv("/content/drive/My Drive/Harvard HW/Course 4 - Final Project/total_movie_dataset.csv", index=False)

movie_dataset

,movie_name,movie_page_url,movie_place,movie_id,movie_thubmnail,movie_metascore,movie_description,runtime_min,movie_vote_num,movie_gross,...,budget_usd,budget_currency,gross_worldwide,gross_us_canada,opening_weekend_us_canada,origin_country,languages,__SuccsefullyCollectExtandedDetails,Director_2_name,Director_2_imdb_id
0,Avatar: The Way of Water,https://www.imdb.com/title/tt1630029/?ref_=adv...,1,tt1630029,https://m.media-amazon.com/images/S/sash/4Fyxw...,67,Jake Sully lives with his newfound family form...,192,338635,"659,682,302",...,350000000.0,USD,"$2,312,335,665","$681,919,456","$134,100,226",United States,English,True,NaN,NaN
1,Everything Everywhere All at Once,https://www.imdb.com/title/tt6710474/?ref_=adv...,2,tt6710474,https://m.media-amazon.com/images/S/sash/4Fyxw...,81,A middle-aged Chinese immigrant is swept up in...,139,415040,"72,861,480",...,14300000.0,USD,"$136,137,373","$77,020,896","$501,305",United States,"English,Mandarin,Cantonese",True,Daniel Scheinert,nm3215397
2,The Whale,https://www.imdb.com/title/tt13833688/?ref_=ad...,3,tt13833688,https://m.media-amazon.com/images/S/sash/4Fyxw...,60,"A reclusive, morbidly obese English teacher at...",117,118671,None,...,10000000.0,USD,"$53,903,396","$17,443,137","$332,152",United States,English,True,NaN,NaN
3,Triangle of Sadness,https://www.imdb.com/title/tt7322224/?ref_=adv...,4,tt7322224,https://m.media-amazon.com/images/S/sash/4Fyxw...,63,A fashion model celebrity couple join an event...,147,118383,None,...,10961391.0,EUR,"$25,400,081","$4,608,096","$214,602",None,"English,Swedish,German,French,Greek,Tagalog,Akan",True,NaN,NaN
4,Top Gun: Maverick,https://www.imdb.com/title/tt1745960/?ref_=adv...,5,tt1745960,https://m.media-amazon.com/images/S/sash/4Fyxw...,78,"After thirty years, Maverick is still pushing ...",130,561836,"718,732,821",...,170000000.0,USD,"$1,493,491,858","$718,732,821","$126,707,459",United States,English,True,NaN,NaN
5,The Banshees of Inisherin,https://www.imdb.com/title/tt11813216/?ref_=ad...,6,tt11813216,https://m.media-amazon.com/images/S/sash/4Fyxw...,87,Two lifelong friends find themselves at an imp...,114,188294,None,...,NaN,NaN,"$48,884,190","$10,582,266","$184,454",None,English,True,NaN,NaN
6,Babylon,https://www.imdb.com/title/tt10640346/?ref_=ad...,7,tt10640346,https://m.media-amazon.com/images/S/sash/4Fyxw...,60,A tale of outsized ambition and outrageous exc...,189,102277,None,...,78000000.0,USD,"$63,377,352","$15,351,455","$3,603,368",United States,"English,Spanish,Italian,Cantonese,French,Hunga...",True,NaN,NaN
7,The Menu,https://www.imdb.com/title/tt9764362/?ref_=adv...,8,tt9764362,https://m.media-amazon.com/images/S/sash/4Fyxw...,71,A young couple travels to a remote island to e...,107,283781,None,...,35000000.0,USD,"$79,628,200","$38,501,125","$9,004,957",United States,"English,Spanish",True,NaN,NaN
8,A Man Called Otto,https://www.imdb.com/title/tt7405458/?ref_=adv...,9,tt7405458,https://m.media-amazon.com/images/S/sash/4Fyxw...,51,Otto is a grump who's given up on life followi...,126,51769,"62,472,236",...,50000000.0,USD,"$109,046,909","$64,246,909","$56,257",None,"English,Spanish",True,NaN,NaN
9,Puss in Boots: The Last Wish,https://www.imdb.com/title/tt3915174/?ref_=adv...,10,tt3915174,https://m.media-amazon.com/images/S/sash/4Fyxw...,73,When Puss in Boots discovers that his passion ...,102,111181,"168,464,485",...,90000000.0,USD,"$479,194,857","$184,933,715","$12,429,515",None,"English,Spanish",True,Januel Mercado,nm2591093


In [ ]:
languages = movie_main_page_soup.find('span',string="Language") if movie_main_page_soup.find('span',string="Languages") is None else movie_main_page_soup.find('span',string="Languages")
languages = [] if not languages else languages.find_next_sibling().find_all('a')
",".join([language.text for language in languages])

'English'

In [ ]:
import requests
from IPython.core.display import HTML
import json
from time import sleep
import re
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import traceback

In [ ]:
movie_dataset = pd.read_csv('/content/drive/My Drive/Harvard HW/Course 4 - Final Project/total_movie_dataset.csv')
movie_dataset

,movie_name,movie_page_url,movie_place,movie_id,movie_thubmnail,movie_metascore,movie_description,runtime_min,movie_vote_num,movie_gross,...,budget_usd,budget_currency,gross_worldwide,gross_us_canada,opening_weekend_us_canada,origin_country,languages,__SuccsefullyCollectExtandedDetails,Director_2_name,Director_2_imdb_id
0,Avatar: The Way of Water,https://www.imdb.com/title/tt1630029/?ref_=adv...,1,tt1630029,https://m.media-amazon.com/images/S/sash/4Fyxw...,67.0,Jake Sully lives with his newfound family form...,192,338635,"659,682,302",...,350000000.0,USD,"$2,312,335,665","$681,919,456","$134,100,226",United States,English,True,NaN,NaN
1,Everything Everywhere All at Once,https://www.imdb.com/title/tt6710474/?ref_=adv...,2,tt6710474,https://m.media-amazon.com/images/S/sash/4Fyxw...,81.0,A middle-aged Chinese immigrant is swept up in...,139,415040,"72,861,480",...,14300000.0,USD,"$136,137,373","$77,020,896","$501,305",United States,"English,Mandarin,Cantonese",True,Daniel Scheinert,nm3215397
2,The Whale,https://www.imdb.com/title/tt13833688/?ref_=ad...,3,tt13833688,https://m.media-amazon.com/images/S/sash/4Fyxw...,60.0,"A reclusive, morbidly obese English teacher at...",117,118671,NaN,...,10000000.0,USD,"$53,903,396","$17,443,137","$332,152",United States,English,True,NaN,NaN
3,Triangle of Sadness,https://www.imdb.com/title/tt7322224/?ref_=adv...,4,tt7322224,https://m.media-amazon.com/images/S/sash/4Fyxw...,63.0,A fashion model celebrity couple join an event...,147,118383,NaN,...,10961391.0,EUR,"$25,400,081","$4,608,096","$214,602",NaN,"English,Swedish,German,French,Greek,Tagalog,Akan",True,NaN,NaN
4,Top Gun: Maverick,https://www.imdb.com/title/tt1745960/?ref_=adv...,5,tt1745960,https://m.media-amazon.com/images/S/sash/4Fyxw...,78.0,"After thirty years, Maverick is still pushing ...",130,561836,"718,732,821",...,170000000.0,USD,"$1,493,491,858","$718,732,821","$126,707,459",United States,English,True,NaN,NaN
5,The Banshees of Inisherin,https://www.imdb.com/title/tt11813216/?ref_=ad...,6,tt11813216,https://m.media-amazon.com/images/S/sash/4Fyxw...,87.0,Two lifelong friends find themselves at an imp...,114,188294,NaN,...,NaN,NaN,"$48,884,190","$10,582,266","$184,454",NaN,English,True,NaN,NaN
6,Babylon,https://www.imdb.com/title/tt10640346/?ref_=ad...,7,tt10640346,https://m.media-amazon.com/images/S/sash/4Fyxw...,60.0,A tale of outsized ambition and outrageous exc...,189,102277,NaN,...,78000000.0,USD,"$63,377,352","$15,351,455","$3,603,368",United States,"English,Spanish,Italian,Cantonese,French,Hunga...",True,NaN,NaN
7,The Menu,https://www.imdb.com/title/tt9764362/?ref_=adv...,8,tt9764362,https://m.media-amazon.com/images/S/sash/4Fyxw...,71.0,A young couple travels to a remote island to e...,107,283781,NaN,...,35000000.0,USD,"$79,628,200","$38,501,125","$9,004,957",United States,"English,Spanish",True,NaN,NaN
8,A Man Called Otto,https://www.imdb.com/title/tt7405458/?ref_=adv...,9,tt7405458,https://m.media-amazon.com/images/S/sash/4Fyxw...,51.0,Otto is a grump who's given up on life followi...,126,51769,"62,472,236",...,50000000.0,USD,"$109,046,909","$64,246,909","$56,257",NaN,"English,Spanish",True,NaN,NaN
9,Puss in Boots: The Last Wish,https://www.imdb.com/title/tt3915174/?ref_=adv...,10,tt3915174,https://m.media-amazon.com/images/S/sash/4Fyxw...,73.0,When Puss in Boots discovers that his passion ...,102,111181,"168,464,485",...,90000000.0,USD,"$479,194,857","$184,933,715","$12,429,515",NaN,"English,Spanish",True,Januel Mercado,nm2591093


## Collect Artist Details

In [ ]:
def assume_gender(artist_bio):
  # Define the pronouns associated with each gender
  male_pronouns = ["he", "him", "his", "himself"]
  female_pronouns = ["she", "her", "hers", "herself"]
  nonbinary_pronouns = ["they", "them", "their", "themselves"]

  # Count the number of male and female pronouns in the bio
  prnoun_count = {'Male':0, "Female":0, "NonBinary":0}

  for word in artist_bio.lower().split():
      if word in male_pronouns:
          prnoun_count["Male"] += 1
      elif word in female_pronouns:
          prnoun_count["Female"] += 1
      elif word in nonbinary_pronouns:
          prnoun_count["NonBinary"] += 1

  # Determine the actor's gender based on the pronoun count
  sorted_d = {k: v for k, v in sorted(prnoun_count.items(), key=lambda item: item[1], reverse=True)}
  Gender = next(iter(sorted_d))

  if prnoun_count[Gender] == 0: # If no prnoune was found
    Gender = "Unknown"

  return Gender

# Extract the bio text
bio_text = "Joseph Vincent Russo is an American filmmaker and producer who works alongside his brother Anthony Russo. They have directed You, Me and Dupree, Cherry and the Marvel films Captain America: The Winter Soldier, Captain America: Civil War, Avengers: Infinity War and Avengers: Endgame. Endgame is one of the highest grossing films of all time."
assume_gender(bio_text)

'UnKnown'

In [ ]:
def extract_artist_info(artist_imdb_id,col_name):
  artist_dict = {}

  artist_url_page = f"{base_url}/name/{artist_imdb_id}"
  artist_dict['artist_imdb_id'] = artist_imdb_id
  artist_dict['artist_type'] = col_name.split('_')[0]
  artist_dict['artist_url'] = artist_url_page

  try:
    artist_page = requests.get(artist_url_page, headers=headers)
    artist_page_soup = BeautifulSoup(artist_page.text, 'html.parser') 

    artist_name = artist_page_soup.find('h1',{'data-testid':'hero__pageTitle'}).find('span').text.strip()
    artist_dict['artist_name'] = artist_name

    artist_rank = artist_page_soup.find('article',{'class':'sc-99c6a4c3-3 gdlLCR'}).find('span').text.strip()

    artist_dict['artist_rank'] = artist_rank
    artist_dict['Is_5000'] = False
    artist_dict['Is_500'] = False
    artist_dict['Is_100'] = False
    artist_dict['Is_10'] = False

    if artist_rank.lower()!="see rank":
      if artist_rank=="Top 5,000":
        artist_dict['Is_5000'] = True
      elif artist_rank=="Top 500":
        artist_dict['Is_500'] = True
        artist_dict['Is_5000'] = True
      else:
        try:
          rank = int(artist_rank)   # Convert the string to an integer
          if rank <= 100:           # Check if the rank is within the top 100
            artist_dict['Is_100'] = True
            artist_dict['Is_500'] = True
            artist_dict['Is_5000'] = True
          if rank <= 10: 
            artist_dict['Is_10'] = True
        except ValueError:
          print(f"Could not interpret value {artist_rank} for artist: {artist_name}. Url: {artist_url_page}")
      
    artist_bio = artist_page_soup.find('div',{'class':'ipc-html-content-inner-div'}).text
    artist_gender = assume_gender(artist_bio)
    artist_dict['artist_gender'] = artist_gender

    artist_dict['award_desc'] = ""
    awards = artist_page_soup.find('div',{'data-testid':'awards'})
    if awards!=None:
      award_desc = awards.find('a').text
      artist_dict['award_desc'] = award_desc

    artist_title = artist_dict['artist_type']
    if artist_title == "Star":
      artist_title = "Actress" if artist_gender=="Female" else "Actor"

    try:
      previous_work = artist_page_soup.find('h3',string=artist_title).find_next().find('li', string="Previous").find_next_sibling().text
    except e:
      previous_work = 0
      print(f"Unable to get previous work for artist: {artist_name}. Url: {artist_url_page}")
    artist_dict['previous_work'] = previous_work
    artist_dict['__SuccsefullyCollectArtistDetails'] = True;

    return artist_dict
  except Exception as e:
    print(f"Failed extracting data for artist: {artist_imdb_id}. Url: {artist_url_page}.\n Error:\n{e}")
    traceback.print_exc()
    artist_dict['__SuccsefullyCollectArtistDetails'] = False;

artist_imdb_id = "nm3215397"
col_name = "Star_3_imdb_id"
extract_artist_info(artist_imdb_id,col_name)

Could not interpret value IMDbProStarmeter for artist: Daniel Scheinert. Url: https://www.imdb.com/name/nm3215397


{'artist_imdb_id': 'nm3215397',
 'artist_type': 'Star',
 'artist_url': 'https://www.imdb.com/name/nm3215397',
 'artist_name': 'Daniel Scheinert',
 'artist_rank': 'IMDbProStarmeter',
 'Is_5000': False,
 'Is_500': False,
 'Is_100': False,
 'Is_10': False,
 'artist_gender': 'Male',
 'award_desc': 'Won 3 Oscars',
 'previous_work': '15',
 '__SuccsefullyCollectArtistDetails': True}

In [ ]:
artists_list = {}

artist_cols = [col for col in movie_dataset.columns if "imdb_id" in col]

for i, row in movie_dataset.iterrows():
    for col_name in artist_cols:
      artist_imdb_id = row[col_name]
      if artist_imdb_id not in artists_list.keys():
        artist_dict = extract_artist_info(artist_imdb_id,col_name)
        artists_list[artist_imdb_id] = artist_dict
        sleep(0.1)
    if i%50 == 0:
      artist_dataset = pd.DataFrame(artists_list)
      artist_dataset.to_csv("/content/drive/My Drive/Harvard HW/Course 4 - Final Project/artists_dataset.csv", index=False)


Could not interpret value IMDbProStarmeter for artist: James Cameron. Url: https://www.imdb.com/name/nm0000116
Could not interpret value IMDbProStarmeter for artist: Rick Jaffa. Url: https://www.imdb.com/name/nm0415425
Could not interpret value IMDbProStarmeter for artist: Amanda Silver. Url: https://www.imdb.com/name/nm0798646
Could not interpret value IMDbProStarmeter for artist: Sam Worthington. Url: https://www.imdb.com/name/nm0941777
Could not interpret value IMDbProStarmeter for artist: Zoe Saldana. Url: https://www.imdb.com/name/nm0757855
Could not interpret value IMDbProStarmeter for artist: Sigourney Weaver. Url: https://www.imdb.com/name/nm0000244
Failed extracting data for artist: nan. Url: https://www.imdb.com/name/nan.
 Error:
'NoneType' object has no attribute 'find'


Traceback (most recent call last):
  File "<ipython-input-90-4f99722bd02a>", line 13, in extract_artist_info
    artist_name = artist_page_soup.find('h1',{'data-testid':'hero__pageTitle'}).find('span').text.strip()
AttributeError: 'NoneType' object has no attribute 'find'


Could not interpret value IMDbProStarmeter for artist: Daniel Kwan. Url: https://www.imdb.com/name/nm3453283
Could not interpret value IMDbProStarmeter for artist: Daniel Scheinert. Url: https://www.imdb.com/name/nm3215397
Failed extracting data for artist: nm3215397. Url: https://www.imdb.com/name/nm3215397.
 Error:
local variable 'e' referenced before assignment


Traceback (most recent call last):
  File "<ipython-input-90-4f99722bd02a>", line 57, in extract_artist_info
    previous_work = artist_page_soup.find('h3',string=artist_title).find_next().find('li', string="Previous").find_next_sibling().text
AttributeError: 'NoneType' object has no attribute 'find_next'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-90-4f99722bd02a>", line 58, in extract_artist_info
    except e:
UnboundLocalError: local variable 'e' referenced before assignment


Could not interpret value IMDbProStarmeter for artist: Michelle Yeoh. Url: https://www.imdb.com/name/nm0000706
Could not interpret value IMDbProStarmeter for artist: Stephanie Hsu. Url: https://www.imdb.com/name/nm3513533
Could not interpret value IMDbProStarmeter for artist: Jamie Lee Curtis. Url: https://www.imdb.com/name/nm0000130
Could not interpret value IMDbProStarmeter for artist: Darren Aronofsky. Url: https://www.imdb.com/name/nm0004716
Could not interpret value IMDbProStarmeter for artist: Samuel D. Hunter. Url: https://www.imdb.com/name/nm5926708
Could not interpret value IMDbProStarmeter for artist: Brendan Fraser. Url: https://www.imdb.com/name/nm0000409
Could not interpret value IMDbProStarmeter for artist: Sadie Sink. Url: https://www.imdb.com/name/nm5584750
Could not interpret value IMDbProStarmeter for artist: Ty Simpkins. Url: https://www.imdb.com/name/nm1339223
Could not interpret value IMDbProStarmeter for artist: Ruben Östlund. Url: https://www.imdb.com/name/nm1128

KeyboardInterrupt: ignored